# Chapter 2: ML Models for Vision

### Factors Making the 5-Flowers Dataset Suboptimal

#### Quantity

- ML models typically require millions of images for training from scratch.
- While there are alternative approaches that can work with fewer images, it's recommended to collect the largest dataset that's practically and ethically possible.

#### Data Format

- Storing images as individual JPEG files is inefficient, as most training time is spent on data reading.
- TensorFlow Record format is recommended for efficiency.

#### Content

- The 5-Flowers dataset consists of found data or images not explicitly collected for the classification task.
- Purposeful data collection should be employed, if allowed by the problem domain.

#### Labeling

- This dataset was manually labeled, which can become impractical for larger datasets.
- Labeling of images is a broad topic and needs careful consideration.

### NOTE:
- your training dataset has to reflect the conditions under which your model will be required to make predictions

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [5]:

import tensorflow as tf
print(tf.version.VERSION)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

2.12.1
Found GPU at: /device:GPU:0


2023-07-29 22:24:56.628141: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 22:24:56.628405: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 22:24:56.628558: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf